In [1]:
from datetime import datetime
import json
import urllib
import urllib2
import re

from bs4 import BeautifulSoup



hdr = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive'
}



# --------------------------------------------- #
def getUrlSoup(url,hdr):
    req1 = urllib2.Request(url, headers=hdr)
    page = urllib2.urlopen(req1)
    content = page.read()
    soup = BeautifulSoup(content)
    return soup
    # --------------------------------------------- #


# --------------------------------------------- #
def getReviews(main_block, reviews_obj=None):
    
    assert reviews_obj is not None
    
    # Treat it as a massive block of text and split it on the horizontal lines.
    reviews_block = re.split(r"<hr[^>]*>", str(main_block))
    
    # Iterate over all the reviews.
    # NOTE: The indexing is dubious here...
    for r in reviews_block[1:-3]:
        
        # Soup the block of html-code-text
        soup_review = BeautifulSoup(r)
        soup_review_header = soup_review.find('div')
        
        # Create an empty dictionary to hold the review.
        review = dict()
        
        # Get the user-name, user-link for the reviewer.
        review['user_name'] = soup_review_header.findAll('a')[1].get_text()
        review['user_href'] = soup_review_header.findAll('a')[1]['href']

        # Try to get the rating.
        soup_imgs = soup_review_header.findAll('img')
        review['review_rating'] = None
        if len(soup_imgs) > 1:
            review['review_rating'] = int(soup_imgs[1]['alt'].split('/')[0])
        
        # Get the review title.
        review['review_title'] = soup_review.find('h2').get_text()
        
        # Get the review content.
        tmp_content = soup_review.findAll('p')
        if len(tmp_content) > 1 and 'spoilers' in tmp_content[0].get_text():
            review['review_content'] = tmp_content[1].get_text()
        else:
            review['review_content'] = tmp_content[0].get_text()
        
        # Get the review usefulness.
        # review['review_useful'] = soup_review_header.find('small').get_text().split('people')[0].rstrip().lstrip()
        
        reviews_obj.append(review)
        
    # Done. Return the reviews_obj
    return reviews_obj
    # --------------------------------------------- #


# --------------------------------------------- #
def fnShowNumberOfReviews(MOVIES):
    s = 0
    for m in MOVIES:
        if 'reviews' in m:
            # print m['name'], len(m['reviews'])
            s += len(m['reviews'])
        else:
            continue

    return s
    # --------------------------------------------- #

Load the list of movies.

In [2]:
MOVIES = json.load(open('../../data-blog---list_of_movies_v3-v1200.json','r'))

print 'We have {} movies in the dataset.'.format(len(MOVIES))

print '\nNumber of reviews available: {}'.format(fnShowNumberOfReviews(MOVIES))

We have 9869 movies in the dataset.

Number of reviews available: 612354


For each movie we will grab the ID and get the reviews.

In [3]:
print 'Grabbing reviews for all movies...\n'

added = 0

# Iterate over all movies.
for e,movie in enumerate(MOVIES[:1400]):
    
    if e == 282 or e == 1314:
        print '\t SKIPPING ---> ', movie['name'],
    
    # Check if the movie already has the reviews populated
    elif 'reviews' not in movie:
        
        print '\t ---> ', movie['name'],
    
        url = 'http://www.imdb.com' + movie['link'] + 'reviews'

        # Get the contents for the first reviews page.
        soup = getUrlSoup(url,hdr)
        # Get the main content block.
        main_block = (soup.body.find('div', attrs={'class':'reviews'})
                               .find('div', attrs={'id':'tn15main'})
                               .find('div', attrs={'id':'tn15content'}))
        # Get the number of pages to get reviews from.
        num_pages = int(main_block.findAll('table')[1]
                                  .find('td',attrs={'nowrap':'1'}).get_text().split(' of ')[1].rstrip(':'))

        MOVIES[e]['num_reviews_pages'] = num_pages
        print ' -- pages: ', num_pages

        # Iterate over all the reviews pages.
        REVIEWS = []

        for i in range(1,num_pages+1):

            # If this is the first page, we don't need to load the new page.
            if i == 1:
                REVIEWS = getReviews(main_block,REVIEWS)

            # We need to grab the contents for the next page.
            else:
                page_soup = getUrlSoup(url + '?start=' + str(10*(i-1)), hdr)
                main_block = (page_soup.body.find('div', attrs={'class':'reviews'})
                                            .find('div', attrs={'id':'tn15main'})
                                            .find('div', attrs={'id':'tn15content'}))
                REVIEWS = getReviews(main_block,REVIEWS)

        # Done for this movie. Add reviews to the movie object.
        MOVIES[e]['reviews'] = REVIEWS
        added += len(REVIEWS)


    # If the review is there. Sanity check...
    # else:
#         # nothing.
#         url = 'http://www.imdb.com' + movie['link'] + 'reviews'
#         soup = getUrlSoup(url,hdr)
#         # Get the main content block.
#         main_block = (soup.body.find('div', attrs={'class':'reviews'})
#                                .find('div', attrs={'id':'tn15main'})
#                                .find('div', attrs={'id':'tn15content'}))
#         # Get the number of pages to get reviews from.
#         num_pages = int(main_block.findAll('table')[1]
#                                   .find('td',attrs={'nowrap':'1'}).get_text().split(' of ')[1].rstrip(':'))
#         if 
        
    if e % 1 == 0 and e > 1000:
        print e, '.',
        

print 'DONE! Added {} reviews'.format(added)

Grabbing reviews for all movies...

	 SKIPPING --->  Till We Meet Again 1001 . 1002 . 1003 . 1004 . 1005 . 1006 . 1007 . 1008 . 1009 . 1010 . 1011 . 1012 . 1013 . 1014 . 1015 . 1016 . 1017 . 1018 . 1019 . 1020 . 1021 . 1022 . 1023 . 1024 . 1025 . 1026 . 1027 . 1028 . 1029 . 1030 . 1031 . 1032 . 1033 . 1034 . 1035 . 1036 . 1037 . 1038 . 1039 . 1040 . 1041 . 1042 . 1043 . 1044 . 1045 . 1046 . 1047 . 1048 . 1049 . 1050 . 1051 . 1052 . 1053 . 1054 . 1055 . 1056 . 1057 . 1058 . 1059 . 1060 . 1061 . 1062 . 1063 . 1064 . 1065 . 1066 . 1067 . 1068 . 1069 . 1070 . 1071 . 1072 . 1073 . 1074 . 1075 . 1076 . 1077 . 1078 . 1079 . 1080 . 1081 . 1082 . 1083 . 1084 . 1085 . 1086 . 1087 . 1088 . 1089 . 1090 . 1091 . 1092 . 1093 . 1094 . 1095 . 1096 . 1097 . 1098 . 1099 . 1100 . 1101 . 1102 . 1103 . 1104 . 1105 . 1106 . 1107 . 1108 . 1109 . 1110 . 1111 . 1112 . 1113 . 1114 . 1115 . 1116 . 1117 . 1118 . 1119 . 1120 . 1121 . 1122 . 1123 . 1124 . 1125 . 1126 . 1127 . 1128 . 1129 . 1130 . 1131 . 1132 . 1133

/Users/luuk/pyEnv/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


AttributeError: 'NoneType' object has no attribute 'get_text'

In [4]:
print 'Number of reviews available: {}'.format(fnShowNumberOfReviews(MOVIES))

 Number of reviews available: 648988


In [5]:
# Full dump
with open('../../data-blog---list_of_movies_v3-v1313.json', 'w') as outfile:
    json.dump(MOVIES, outfile)

In [ ]:
# # Dump JSONL line by line
# with open('../../list_of_movies_with_reviews.json', 'a') as the_file:
# the_file.write('Hello\n')